In [2]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

print(train_np_list[0])

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], [68, 61, 59, 54], [69, 66, 61, 54], [69, 66, 61, 56], [69, 66, 61, 57], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 59, 49], [68, 65, 59, 49], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 59, 50], [66, 66, 59, 50], [68, 66, 59, 50], [68, 66, 59, 50], [69, 66, 61, 49], [69, 66, 61, 49], [69, 66, 61, 47], [69, 66, 61, 47], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 59, 49], [68, 65, 59, 49], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [71, 66, 59, 56], [71, 66, 59, 56], [71, 65, 59, 56], [71, 65, 59, 56], [69, 66, 61, 57], [69, 66, 61, 57], [69, 66, 61, 59], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [69, 66, 61, 54], [69, 66, 61, 54], [69, 66, 61, 52], [69, 66, 

In [14]:
def build_dataset(chorales):
    def reshape(chorale):
        chorale = tf.reshape(chorale.to_tensor(), [-1])
        return chorale
    def target(chorale):
        X = chorale[:-1]
        Y = chorale[1:]
        return X, Y
    
    ragged_chorales = tf.ragged.constant(chorales)
    dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
    dataset = dataset.map(reshape)
    dataset = dataset.map(target)
    for d in dataset:
        tf.print(d)
        
build_dataset(train_np_list)

([66 61 57 ... 66 61 57], [61 57 54 ... 61 57 54])
([64 59 55 ... 71 63 54], [59 55 52 ... 63 54 47])
([69 65 62 ... 62 57 54], [65 62 50 ... 57 54 50])
([62 57 53 ... 62 57 54], [57 53 50 ... 57 54 38])
([67 62 59 ... 67 62 59], [62 59 55 ... 62 59 43])
([74 71 67 ... 67 62 59], [71 67 55 ... 62 59 43])
([74 69 65 ... 74 66 57], [69 65 62 ... 66 57 50])
([69 64 61 ... 69 64 61], [64 61 45 ... 64 61 45])
([69 64 61 ... 69 64 61], [64 61 57 ... 64 61 45])
([71 67 64 ... 64 59 56], [67 64 52 ... 59 56 52])
([63 58 55 ... 63 58 55], [58 55 51 ... 58 55 39])
([71 66 62 ... 66 62 57], [66 62 59 ... 62 57 50])
([69 64 60 ... 69 61 52], [64 60 45 ... 61 52 45])
([73 69 64 ... 69 61 52], [69 64 57 ... 61 52 45])
([71 64 56 ... 64 59 56], [64 56 50 ... 59 56 52])
([67 62 58 ... 67 59 50], [62 58 55 ... 59 50 43])
([71 66 62 ... 71 66 62], [66 62 59 ... 66 62 47])
([67 62 58 ... 74 66 57], [62 58 43 ... 66 57 50])
([69 66 62 ... 62 57 54], [66 62 50 ... 57 54 50])
([64 60 57 ... 64 59 56], [60 5